In [ ]:
"""
This script processes feature changes in a Twitter user dataset and calculates various statistical measures based on Levenshtein distance between previous and current values. It stores the processed data in a SQLite database.

The process_feature_changes function calculates the Levenshtein distance for each feature change and computes statistical measures such as maximum, minimum, median, average, standard deviation, variance, mode, edit distance per character, skewness, range, median absolute deviation, and coefficient of variation.

The script connects to the database, defines the necessary tables, retrieves the user IDs, and processes the feature changes in chunks. It then inserts the processed data into the 'processed_data' table.

Please make sure to update the database connection details and adjust the features and new_features lists according to your dataset.

"""

import sqlite3
from Levenshtein import distance as levenshtein_distance
import statistics
import datetime
from scipy.stats import skew
import numpy as np

# define function to process feature changes
def process_feature_changes(rows, feature_name):
    user_changes = {}
    for row in rows:
        user_id = row['user_id']
        prev_value, curr_value = row['previous'].strip(), row['current'].strip()
        user_changes.setdefault(user_id, []).append(levenshtein_distance(prev_value, curr_value))
        num_changes = len(user_changes[user_id]) if user_changes[user_id] else 0

    return {user_id: {
                "num_changes": num_changes,
                "max_edit_distance": max(user_changes[user_id], default=-1),
                "min_edit_distance": min(user_changes[user_id], default=-1),
                "median_edit_distance": statistics.median(user_changes[user_id]) if user_changes[user_id] else -1,
                "average_edit_distance": statistics.mean(user_changes[user_id]) if user_changes[user_id] else -1,
                "std_edit_distance": statistics.stdev(user_changes[user_id]) if num_changes > 1 else 0 if num_changes == 1 else -1,
                "var_edit_distance": statistics.variance(user_changes[user_id]) if num_changes > 1 else 0 if num_changes == 1 else -1,
                "mode_edit_distance": statistics.mode(user_changes[user_id]) if user_changes[user_id] else -1,
                "edit_distance_per_character": statistics.mean(user_changes[user_id]) / (
                    (len(prev_value) + len(curr_value)) / 2) if user_changes[user_id] else -1,
                "skew_edit_distance": skew(user_changes[user_id], bias=False) if num_changes > 1 and statistics.stdev(user_changes[user_id]) != 0 else 100,
                "range_edit_distance": max(user_changes[user_id], default=-1) - min(user_changes[user_id], default=-1),
                "mad_edit_distance": statistics.median(
                    [abs(x - statistics.median(user_changes[user_id])) for x in user_changes[user_id]]) if user_changes[
                    user_id] else 0 if num_changes == 1 else -1,
                "cv_edit_distance": statistics.stdev(user_changes[user_id]) / statistics.mean(user_changes[user_id]) if num_changes > 1 else 0 if len(user_changes[user_id]) == 1 else -1,
            }
            for user_id in user_changes}

# connect to database
conn = sqlite3.connect('TwitterUserChanges.db')
conn.row_factory = sqlite3.Row

# create table if not exists
conn.execute("""
    CREATE TABLE IF NOT EXISTS processed_data (
        user_id TEXT,
        feature_name TEXT,
        value REAL,
        test_time TEXT,
        UNIQUE(user_id, feature_name)
    )
""")

# get current time
current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# define features and new_features
features = ["full_name", "username", "description"]
new_features = [
"num_changes",
"max_edit_distance",
"min_edit_distance",
"median_edit_distance",
"average_edit_distance",
"std_edit_distance",
"var_edit_distance",
"mode_edit_distance",
"edit_distance_per_character",
"skew_edit_distance",
"range_edit_distance",
"mad_edit_distance",
"cv_edit_distance",
]

# get distinct user IDs
cursor = conn.execute("SELECT DISTINCT user_id FROM users")
user_ids = [row[0] for row in cursor.fetchall()]


# Define the chunk size for batch insertion
chunk_size = len(user_ids)
chunk_amount = len(user_ids)/chunk_size

# Loop through user IDs in chunks
for i in range(0, len(user_ids), chunk_size):
    user_ids_chunk = user_ids[i:i+chunk_size]
    remaining_iterations2 = chunk_size

    # Get all the rows for the user_ids_chunk for all features at once
    rows = conn.execute("SELECT user_id, previous, current, feature FROM changes WHERE user_id IN ({}) AND feature IN ({})".format(','.join('?' for _ in user_ids_chunk), ','.join('?' for _ in features)), tuple(user_ids_chunk + features)).fetchall()

    # Group the rows by user_id and feature
    grouped_rows = {}
    for row in rows:
        grouped_rows.setdefault(row['user_id'], {}).setdefault(row['feature'], []).append(row)

    # Create an array to store processed data for the chunk of users
    processed_data_chunk = np.zeros((chunk_size*len(features)*len(new_features), 3), dtype=np.object)
    j = 0

    # Loop through each user in the chunk
    for user_id in user_ids_chunk:
        # Loop through each feature for the user
        for k, feature in enumerate(features):
            # Get the rows for the user_id and feature
            rows = grouped_rows.get(user_id, {}).get(feature, [])

            # Process the feature changes and get the values for new features
            processed_feature_changes = process_feature_changes(rows, feature)

            # Insert the values into the processed data array
            for l, stat_name in enumerate(new_features):
                value = processed_feature_changes.get(user_id, {}).get(stat_name, -1 if stat_name != "num_changes" and stat_name != "skew_edit_distance" else 0 if stat_name == "num_changes" else 100 if stat_name == "skew_edit_distance" else None)
                processed_data_chunk[j, 0] = user_id
                processed_data_chunk[j, 1] = "Basic " + feature + " " + stat_name
                processed_data_chunk[j, 2] = value
                j += 1
                if j == len(new_features) :
                    break

        remaining_iterations2-=1
        print(("Iterations left:", remaining_iterations2))
        if j == chunk_size*len(features)*len(new_features):
            break

    # Insert the chunk of data into the database
    cursor = conn.cursor()
    # cursor.executemany("INSERT OR REPLACE INTO processed_data (user_id, feature_name, value, test_time) VALUES (?, ?, ?, ?)", [(row[0], row[1], row[2], current_time) for row in processed_data_chunk if any(row)])
    values = [(row[0], row[1], row[2], current_time) for row in processed_data_chunk if any(row)]
    cursor.executemany(
        "INSERT OR REPLACE INTO processed_data (user_id, feature_name, value, test_time) VALUES (?, ?, ?, ?)", values)

    conn.commit()

    chunk_amount -= 1
    print("chunk_left:", chunk_amount)

# Close the connection to the database
conn.close()